In [ ]:
#Installing Prerequired Libraries
!pip install -U transformers datasets accelerate sentencepiece


In [ ]:
import transformers
print(transformers.__version__)


4.57.0


In [ ]:
import torch
print(torch.cuda.is_available())


True


In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"


 **Summarization Base MOdel**

In [ ]:
os.environ["WANDB_MODE"] = "disabled"

import os
import torch
from datasets import load_dataset
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)

# ===========================================
# CONFIGURATION
# ===========================================
BASE_SUM_MODEL = os.getenv("BASE_SUM_MODEL", "t5-base")  # Larger model
DATA_DIR = "data"      # Folder containing *_summarization.jsonl
OUTPUT_DIR = "models/t5_base_summarizer"

MAX_SOURCE_LENGTH = 512
MAX_TARGET_LENGTH = 128
BATCH_SIZE = 2          # Reduce if you get CUDA OOM
EPOCHS = 3
LEARNING_RATE = 5e-5

# ===========================================
# DEVICE SETUP
# ===========================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n🧠 Using device: {device}")
print(f"📘 Base Model: {BASE_SUM_MODEL}")

# ===========================================
# LOAD DATASETS
# ===========================================
train_path = os.path.join(DATA_DIR, "train_summarization.jsonl")
val_path = os.path.join(DATA_DIR, "val_summarization.jsonl")

data_files = {
    "train": train_path,
    "validation": val_path
}

dataset = load_dataset("json", data_files=data_files)
print(f"✅ Dataset loaded: {dataset}")

# ===========================================
# TOKENIZER & MODEL
# ===========================================
tokenizer = T5Tokenizer.from_pretrained(BASE_SUM_MODEL)
model = T5ForConditionalGeneration.from_pretrained(BASE_SUM_MODEL).to(device)

# ===========================================
# PREPROCESS FUNCTION
# ===========================================
def preprocess_function(examples):
    inputs = examples["text"]
    targets = examples["summary"]

    model_inputs = tokenizer(
        inputs, max_length=MAX_SOURCE_LENGTH, truncation=True, padding="max_length"
    )

    labels = tokenizer(
        targets, max_length=MAX_TARGET_LENGTH, truncation=True, padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize datasets
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["text", "summary", "id"]
)

print(f"🧾 Tokenization complete! Sample keys: {list(tokenized_datasets['train'].features.keys())}")

# ===========================================
# TRAINER SETUP
# ===========================================
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# training_args = Seq2SeqTrainingArguments(
#     output_dir=OUTPUT_DIR,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=LEARNING_RATE,
#     per_device_train_batch_size=BATCH_SIZE,
#     per_device_eval_batch_size=BATCH_SIZE,
#     weight_decay=0.01,
#     save_total_limit=2,
#     num_train_epochs=EPOCHS,
#     predict_with_generate=True,
#     fp16=torch.cuda.is_available(),  # Enable mixed precision on GPU
#     logging_dir=f"{OUTPUT_DIR}/logs",
#     logging_steps=50,
#     report_to="none"
# )

training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir='./logs',
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# ===========================================
# TRAIN MODEL
# ===========================================
print("\n🚀 Training started with t5-base...\n")
trainer.train()

# ===========================================
# SAVE MODEL
# ===========================================
print("\n✅ Training completed. Saving model and tokenizer...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"📂 Model saved to: {OUTPUT_DIR}")



🧠 Using device: cuda
📘 Base Model: t5-base
✅ Dataset loaded: DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'summary'],
        num_rows: 3518
    })
    validation: Dataset({
        features: ['id', 'text', 'summary'],
        num_rows: 439
    })
})


Map:   0%|          | 0/3518 [00:00<?, ? examples/s]

Map:   0%|          | 0/439 [00:00<?, ? examples/s]

🧾 Tokenization complete! Sample keys: ['input_ids', 'attention_mask', 'labels']

🚀 Training started with t5-base...



/tmp/ipython-input-940054281.py:115: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saurabh-22210060 (trials_) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.366200
1000,0.000100
1500,0.000000
2000,0.000000
2500,0.000000
3000,0.000000
3500,0.000000
4000,0.000000
4500,0.000000
5000,0.000000



✅ Training completed. Saving model and tokenizer...
📂 Model saved to: models/t5_base_summarizer


**Evaluate Summarizer Model Performance**

In [ ]:
pip install --upgrade transformers datasets


**DistilBert Model For Labeling And Risk**

In [ ]:
import os
import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support

# ==============================
# Config
# ==============================
DATA_DIR = 'data'
TRAIN_FILE = os.path.join(DATA_DIR, 'train_clf.jsonl')
VAL_FILE = os.path.join(DATA_DIR, 'val_clf.jsonl')
OUTPUT_DIR = 'models/distilbert_risk_multilabel'
BASE_MODEL = 'distilbert-base-uncased'  # You can change to any compatible model

# ==============================
# Metrics for multi-label classification
# ==============================
def compute_metrics(pred):
    logits = pred.predictions
    probs = 1 / (1 + np.exp(-logits))  # sigmoid
    preds = (probs >= 0.5).astype(int)
    labels = pred.label_ids
    p, r, f, _ = precision_recall_fscore_support(labels, preds, average='micro', zero_division=0)
    return {'precision': float(p), 'recall': float(r), 'f1': float(f)}

# ==============================
# Preprocessing function
# ==============================
def preprocess(examples, tokenizer):
    tokens = tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',  # Use 'longest' if memory is limited
        max_length=512
    )
    # Convert labels to float32
    tokens['labels'] = np.array(examples['labels'], dtype=np.float32)
    return tokens

# ==============================
# Custom collate function (optional for float labels)
# ==============================
def collate_fn(batch):
    return {
        'input_ids': torch.stack([torch.tensor(x['input_ids']) for x in batch]),
        'attention_mask': torch.stack([torch.tensor(x['attention_mask']) for x in batch]),
        'labels': torch.stack([torch.tensor(x['labels'], dtype=torch.float32) for x in batch])
    }

# ==============================
# Main training function
# ==============================
def main():
    # Load dataset
    dataset = load_dataset('json', data_files={'train': TRAIN_FILE, 'val': VAL_FILE})

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
    sample = next(iter(dataset['train']))
    num_labels = len(sample['labels'])
    model = AutoModelForSequenceClassification.from_pretrained(
        BASE_MODEL,
        num_labels=num_labels,
        problem_type='multi_label_classification'
    )

    # Preprocess dataset
    tokenized = dataset.map(lambda x: preprocess(x, tokenizer), batched=True, remove_columns=dataset['train'].column_names)

    # Training arguments (compatible with older transformers)
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        learning_rate=2e-5,
        weight_decay=0.01,
        logging_steps=50
        # evaluation_strategy, save_strategy removed for old versions
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized['train'],
        eval_dataset=tokenized['val'],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        data_collator=collate_fn  # ensures labels are float32
    )

    # Train
    trainer.train()

    # Evaluate manually
    results = trainer.evaluate()
    print("Validation results:", results)

    # Save model
    trainer.save_model(OUTPUT_DIR)
    print(f"✅ Model saved to {OUTPUT_DIR}")

# ==============================
# Run
# ==============================
if __name__ == '__main__':
    main()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3518 [00:00<?, ? examples/s]

Map:   0%|          | 0/439 [00:00<?, ? examples/s]

/tmp/ipython-input-3822362769.py:85: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.563900
100,0.553700
150,0.553900
200,0.509300
250,0.519000
300,0.534500
350,0.519300
400,0.545700
450,0.506300
500,0.499500


Validation results: {'eval_loss': 0.5004199743270874, 'eval_precision': 0.7839403973509934, 'eval_recall': 0.85623869801085, 'eval_f1': 0.8184961106309421, 'eval_runtime': 9.6886, 'eval_samples_per_second': 45.311, 'eval_steps_per_second': 5.677, 'epoch': 3.0}
✅ Model saved to models/distilbert_risk_multilabel


**Evaluation Metrics Of Model**

In [ ]:
import os

print(os.listdir("models/t5_base_summarizer"))
print(os.listdir("models/roberta_risk_multilabel"))


['checkpoint-5000', 'added_tokens.json', 'tokenizer_config.json', 'spiece.model', 'model.safetensors', 'config.json', 'special_tokens_map.json', 'generation_config.json', 'training_args.bin', 'checkpoint-5277']
['runs']


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSequenceClassification
import torch
import nltk

nltk.download("punkt")
from nltk.tokenize import sent_tokenize

# ---- Load LOCAL models ----
summ_model_name = "models/t5_base_summarizer"        # ✅ Local folder path (contains config.json etc.)
clf_model_name = "models/distilbert_risk_multilabel"    # ✅ Local folder path for classifier

# ---- Load models and tokenizers ----
summ_tokenizer = T5Tokenizer.from_pretrained(summ_model_name)
summ_model = T5ForConditionalGeneration.from_pretrained(summ_model_name)

clf_tokenizer = AutoTokenizer.from_pretrained(clf_model_name)
clf_model = AutoModelForSequenceClassification.from_pretrained(clf_model_name)

# Example label names (update to yours)
LABELS = ["Financial", "Legal", "Operational", "Data Privacy", "Reputational"]

# ---- Helper functions ----
def summarize_text(text):
    input_ids = summ_tokenizer(
        "summarize: " + text, return_tensors="pt",
        truncation=True, padding="longest"
    ).input_ids
    output = summ_model.generate(input_ids, max_length=200, num_beams=4, early_stopping=True)
    return summ_tokenizer.decode(output[0], skip_special_tokens=True)

def predict_risk_labels(text):
    inputs = clf_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = clf_model(**inputs)
    probs = torch.sigmoid(outputs.logits).squeeze().tolist()
    return {label: round(prob, 3) for label, prob in zip(LABELS, probs)}

# ---- Run on sample text ----
sample_text = """
The vendor failed to meet the data protection requirements and delayed payments for two consecutive quarters.
"""

summary = summarize_text(sample_text)
risks = predict_risk_labels(sample_text)

print("📝 Summary:\n", summary)
print("\n⚠️ Risk Scores:")
for label, score in risks.items():
    print(f"{label}: {score}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


📝 Summary:
 the vendor failed to meet the data protection requirements and delayed payments for two consecutive quarters .

⚠️ Risk Scores:
Financial: 0.536
Legal: 0.504
Operational: 0.739
Data Privacy: 0.657


**Download Model**  

In [3]:
!zip -r t5_base_summarizer.zip /models/t5_base_summarizer
!zip -r distilbert_risk_multilabel.zip /models/distilbert_risk_multilabel

from google.colab import files
files.download("t5_base_summarizer.zip")
files.download("distilbert_risk_multilabel.zip")


	zip warning: name not matched: /models/t5_base_summarizer

zip error: Nothing to do! (try: zip -r t5_base_summarizer.zip . -i /models/t5_base_summarizer)
	zip warning: name not matched: /models/distilbert_risk_multilabel

zip error: Nothing to do! (try: zip -r distilbert_risk_multilabel.zip . -i /models/distilbert_risk_multilabel)
